# Packages

In [1]:
# Questions require data manipulation (given csv file as well), so import the two most used packages
import pandas as pd
import numpy as np
# os just to make my life easier when it comes to reading data and saving to places :)
import os

# Question 1

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

1. Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
2. What metric would you report for this dataset?
3. What is its value?


In [2]:
# Read the data
shoe_data = pd.read_csv(os.path.join('data', 'sneaker_data.csv'))

shoe_data.sample(n=5)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
3791,3792,11,714,184,1,debit,2017-03-07 22:48:35
3736,3737,22,829,292,2,cash,2017-03-01 4:45:25
877,878,51,739,374,2,cash,2017-03-09 20:35:27
3303,3304,99,904,390,2,cash,2017-03-30 17:52:05
4541,4542,70,724,519,3,credit_card,2017-03-24 17:18:23


Taking a quick glance at the data, there's a bunch of columns. The ones that seem to be helpful for the problem at hand are:
- shop_id to identify the unique stores
- order_id to identify each unique order 
- order_amount and total_items to help calculate the AOV
- created_at to figure out 30 day window

Sanity Check:
- are there 100 stores?
- which 30 day window to consider?
- are the order_ids unique? 


In [3]:
# Check if there are exactly 100 stores
np.unique(shoe_data.shop_id)

# exactly 100 ids so there are exactly 100 stores

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])

In [4]:
# Check the times
shoe_data.created_at.sort_values()

# Since the oldest and most recent dates are within the 2017-03, use the entire datasets for the 30 day window

1862    2017-03-01 0:08:09
1741    2017-03-01 0:10:19
3228    2017-03-01 0:14:12
1267    2017-03-01 0:19:31
2689    2017-03-01 0:22:25
               ...        
2765    2017-03-30 9:22:41
4890    2017-03-30 9:27:00
244     2017-03-30 9:30:28
211     2017-03-30 9:40:40
145     2017-03-30 9:55:00
Name: created_at, Length: 5000, dtype: object

In [5]:
# Check uniqueness of order_id

len(np.unique(shoe_data.order_id)) 

#exactly 5000 unique ids out of 5000 orders

5000

All concerns verified

## Q1.1/Q1.3:

Given that AOC = Revenue/Number of Orders, to get such a high number, either the number of orders could have been underestimated or the revenue could be overestimated.

As the calculations were naively done, there are a few things to consider:

1. the data has a total_items section, so it's possible the calculation of $3145.13 was done without taking consideration of that column. Hence, it would seem like someone bought sneakers for a large amount (for example, order_id 2044, $764 for 1 sneaker, if read incorrectly)
2. the order_amount could be incorrect due to possible outliers or wrong input data

Overall, the first reason is the most likely culprit.

In [6]:
# Confirm Naive approach

print(sum(shoe_data.order_amount)/shoe_data.shape[0])

# Correct approach

print(sum(shoe_data.order_amount)/sum(shoe_data.total_items))



3145.128
357.92152221412965


This shows that the first reason was the reason for the naive calculation.

The actual AOC value is $357.92

## Q1.2

The dataset given should be used to determine the impact that each individual store has on shoe sales. If a store is doing extremely well, it is a good sign to keep the store up and potentially think of ways to further boost the popularity of said store. If a store is doing poorly, more attention should be brought to that store to see what the root cause is. The AOC metric does not clearly show the impact of individual stores as it is a single number that represents 100 stores and the fact that it is an average. The average metric has a tendency to do poorly when outliers exist (hence it is not a robust metric). The best metric would be to find the AOC per UNIQUE STORE. Especially in this unique case, outliers will not exist within each store since they only sell a single pair of sneakers.

NOTE: If a single unit of measurement is required as a metric, using the median of the AOC per unique store would be much better than the average (since median is more robust to outliers)

In [7]:
plot_data = shoe_data.groupby(by='shop_id').sum()

plot_data['AOC'] = plot_data.order_amount/plot_data.total_items

plot_data = plot_data.sort_values(by = 'AOC')


print(plot_data.AOC.median())

plot_data



153.0


,order_id,user_id,order_amount,total_items,AOC
shop_id,,,,,
92,102864,36334,6840,76,90.0
2,126448,47370,9588,102,94.0
32,96561,35986,7979,79,101.0
100,94261,34093,8547,77,111.0
7,145326,47441,12208,109,112.0
...,...,...,...,...,...
99,128844,45693,18330,94,195.0
89,172859,50618,23128,118,196.0
12,135437,44755,18693,93,201.0


As suspected, there is a huge outlier when it comes to shop_id 78 (even shop 42 seems to be an outlier). So there is either a misinput on their end or they are selling not-so-affordable shoes! Using the median approach, the median AOC is $153 which seems more reasonable than the overall AOC of $357.92 (without removing anything).

# Question 2: 
For this question you’ll need to use SQL. Follow this link to access the data set required for the challenge. Please use queries to answer the following questions. Paste your queries along with your final numerical answers below.

1. How many orders were shipped by Speedy Express in total?
2. What is the last name of the employee with the most orders?
3. What product was ordered the most by customers in Germany?


## Q2.1

Answer: 54

In [9]:
/* Key words:

    Orders 
    Speedy Express being the shipper
    Total orders

The database Shipper has the relationship between ShipperID and ShipperName
The database Orders has the relationship between ShipperID and Number of Unique Orders (once calculated)

So the goal should be to join the two databases such that the final result contains ShipperName and Number of Unique Orders

*/


SELECT COUNT(OrderID), ShipperName
FROM Orders
INNER JOIN Shippers
WHERE Orders.ShipperID == Shippers.ShipperID
GROUP BY ShipperName;

SyntaxError: invalid syntax (<ipython-input-9-e1de8d0c7e85>, line 1)

## Q2.2

Answer: Peacock

In [10]:
/* Key words:

    Employee
    Orders (most)

The database Employees has the relationship between EmployeeID and LastName
The database Orders has the relationship between EmployeeID and Number of Orders

Goal:

    Join the two database together on Employee and then count the number of orders based off EmployeeID and lastname.

*/

SELECT COUNT(OrderID) TotalOrders, Orders.EmployeeID, Employees.LastName
FROM Orders
INNER JOIN Employees
WHERE Orders.EmployeeID == Employees.EmployeeID
GROUP BY Orders.EmployeeID
ORDER BY TotalOrders;

SyntaxError: invalid syntax (<ipython-input-10-e0f470ba299e>, line 1)

## Q2.3

Answer: Boston Crab Meat

In [11]:
/* Key words:

    Most ordered Product
    Germany

The database Customers has the relationship between Country (where we need Germany for Country) and CustomerID.
The database Orders has the relationship between CustomerID and OrderID
The database OrderDetails has the relationship between OrderID and Quantity and ProductID
The database Products has the relationship between ProductID and ProductName


Goal:

    There needs to be three inner joins to make one master table that has all the information required to calculate the most 
    popular item ordered from Germany.

    Aggregating on the master table and ordering by size should return a table of:
        TotalQuantity, ProductName, ProductID (if the name is not unique for some reason)

*/

SELECT SUM(Quantity) as TotalQuantity, P.ProductID, P.ProductName
FROM Customers as C
INNER JOIN Orders as O
ON C.CustomerID == O.CustomerID
INNER JOIN OrderDetails as OD
ON O.OrderID == OD.OrderID
INNER JOIN Products as P
ON OD.ProductID == P.ProductID
WHERE Country == 'Germany'
GROUP BY OD.ProductID
ORDER BY TotalQuantity;

SyntaxError: invalid syntax (<ipython-input-11-b85d1799b8c6>, line 1)